In [105]:
import torch
import torch.nn.functional as F
import random

g = torch.Generator().manual_seed(1337)

In [2]:
class Linear:
  def __init__(self, ins, outs, bias=False):
    self.weights = torch.empty(outs, ins)
    torch.nn.init.kaiming_uniform_(self.weights, mode='fan_in', nonlinearity='relu', generator=g)
    if bias:
      self.biases = torch.rand(outs, generator=g)
    else:
      self.biases = None

  def __call__(self, x):
    pre_act = x @ self.weights.T
    if self.biases is not None:
      pre_act += self.biases
    return pre_act

  def params(self):
    return [self.weights] + [self.biases] if self.biases is not None else [self.weights]

In [6]:
class Relu:
  def __call__(self, x):
    self.out = F.relu(x)
    return self.out

In [7]:
class BatchNorm1d:
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.dim = dim
    self.eps = eps
    self.momentum = momentum
    self.training = True
    # gamma & beta: trainable parameters to scale or move the normed batch
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
    # for inference
    self.running_mean = torch.zeros(dim)
    self.running_var = torch.ones(dim)

  def __call__(self, x):
    if self.training:   # train vars
      xmean = x.mean(0, keepdim=True)
      xvar = x.var(0, keepdim=True)
    else:               # inference vars
      xmean = self.running_mean
      xvar = self.running_var

    xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
    self.out = self.gamma * xhat + self.beta

    if self.training: # calculate running mean/var
      self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
      self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
    return self.out

  def params(self):
    return [self.gamma] + [self.beta]

In [ ]:
# ---------------------------------------------------------------------

In [5]:
n_hidden = 100 #neuron count for hidden layers


layers = [
    Linear(26)
]

tensor([[-1.1927, -0.2194,  0.2408, -0.6802, -1.2519, -0.7259,  0.0289, -0.4021],
        [-0.0124, -0.8474,  0.5563,  0.5899,  0.7548,  0.2843, -0.2643,  1.1209],
        [ 0.3482, -1.3332, -0.9160,  0.2289,  1.0687,  0.5881,  1.0930,  1.3121]])

In [83]:
with open('turkish_names.txt', 'r') as f:
  lines = f.read().splitlines()
len(lines)

trchr_to_utf8 = {
    'İ': 'i',
    'I': '!',
    'Ö': '@',
    'Ü': '#',
    'Ş': '$',
    'Ç': '^',
    'Ğ': '&'}

words = [word for line in lines for word in line.split()]
names = [''.join(trchr_to_utf8.get(ch, ch) for ch in name).lower() for name in words]
# ^ turkish phonemes transmogrified into utf-8 sigils ^-^
names[:10]

['jale',
 'ali',
 'mahmut',
 'mansur',
 'k#r$ad',
 'gamze',
 'mira^',
 'y#cel',
 'kubilay',
 'hayati']

In [93]:
chars = ['.'] + sorted(list(set(ch for name in names for ch in name)))
# ^ no need to sort tbh.    '.' as special start/end token.
i_to_s = {i: chars[i] for i in range(len(chars))}
s_to_i = {v: k for k, v in i_to_s.items()}

In [116]:
block_size = 3 # how many characters should it take to predict the next one?
# . . . -> a; . . a -> t; . a t -> a
def create_dataset(names):
  X, Y = [], []
  for name in names:
    name = f"{name}."   # add end token to each name
    context = [0] * block_size # initialized as `. . .`
    for ch in name:
      X.append(context)
      Y.append(s_to_i[ch])
      context = context[1:] + [s_to_i[ch]]
  return torch.tensor(X), torch.tensor(Y)

In [148]:
random.shuffle(names)

n1 = int(len(names) * 0.1)

Xtr, Ytr = create_dataset(names[n1*2:]) # 80% train split
Xte, Yte = create_dataset(names[:n1]) # 10% test
Xval, Yval = create_dataset(names[n1:n1*2]) # 10% valid

# precision spell
avg_len = sum(len(name) + 1 for name in names) / len(names)
actual = (len(Xtr) + len(Xte) + len(Xval)) / len(names)
assert abs(avg_len - actual) < 1e-6, '\nSize mismatch.\nWhat did you break?'

In [ ]:
# steps for tomorrow:
# create an embedding table. we should be able to plug in our indices corresponding to characters,
# and get an embedding of them.

# we will backpropagate through the embeddings.